In [32]:
from tools.functions import *
import glob

In [33]:
# chose the conditions to plot

condition = "sparse_age_01_born"
path = "output_final/"

list_condition = glob.glob(path + "*" + condition + ".pkl")
print("Found " + str(len(list_condition)) + " matching files:")

Found 3 matching files:


In [34]:
off_time = 1
threshold = [100, 150, 200] # not used if binarize = False
binarize = False # should be false
normalize = True # should be true 

aggregated_outputs = {}
switch = True

for version in list_condition: 
    output = load_output(version, path="", extension="")
    Mv_loaded = output["Mv"]
    Mh_loaded = output["Mh"]
    time_test_on = output["time_test_on"]
    time_points_dict = output["time_points_dict"]
    n_seed_patterns = output["n_seed_patterns"]
    n_prototype_per_seed = output["n_prototype_per_seed"]
    n_variations_per_prototype = output["n_variations_per_prototype"]
    after_split_n_per_prototype_test = output["after_split_n_per_prototype_test"]

    hamming_distances, percent_match, originals, recovered = hamming_distances_test(Mv_loaded, time_test_on, time_points_dict, off_time=off_time, normalize=normalize,
                                                                                    binarize = binarize, threshold = threshold)
    hd_table_between = np.array(hamming_distances[:10 * after_split_n_per_prototype_test * n_seed_patterns]).reshape(10, after_split_n_per_prototype_test * n_seed_patterns)
    pm_table_between = np.array(percent_match[:10 * after_split_n_per_prototype_test * n_seed_patterns]).reshape(10, after_split_n_per_prototype_test * n_seed_patterns)
    hd_table_within = np.array(hamming_distances[10 * after_split_n_per_prototype_test * n_seed_patterns:]).reshape(10, after_split_n_per_prototype_test * n_seed_patterns)
    pm_table_within = np.array(percent_match[10 * after_split_n_per_prototype_test * n_seed_patterns:]).reshape(10, after_split_n_per_prototype_test * n_seed_patterns)
    
    if switch == True: 
        aggregated_outputs["hd_table_between"] = hd_table_between
        aggregated_outputs["pm_table_between"] = pm_table_between
        aggregated_outputs["hd_table_within"] = hd_table_within
        aggregated_outputs["pm_table_within"] = pm_table_within
        switch = False
    else:
        aggregated_outputs["hd_table_between"] = np.concatenate((aggregated_outputs["hd_table_between"], hd_table_between), axis=1)
        aggregated_outputs["pm_table_between"] = np.concatenate((aggregated_outputs["pm_table_between"], pm_table_between), axis=1)
        aggregated_outputs["hd_table_within"] = np.concatenate((aggregated_outputs["hd_table_within"], hd_table_within), axis=1)
        aggregated_outputs["pm_table_within"] = np.concatenate((aggregated_outputs["pm_table_within"], pm_table_within), axis=1)


# save the aggregated outputs

saving_path = "final_data/"

import pickle
with open(saving_path + condition + "_aggregated_outputs.pkl", 'wb') as f:
    pickle.dump(aggregated_outputs, f)


